## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
from pypdf import PdfReader
import gradio as gr

In [16]:
load_dotenv(override=True)
import os
os.environ["OPENAI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["OPENAI_BASE_URL"] = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI()

In [17]:
print(linkedin)

   
Contact
mahadrana054@gmail.com
www.linkedin.com/in/mahad-
rana-21a504378 (LinkedIn)
Top Skills
Python (Programming Language)
Mahad Rana
Aspiring LLM Engineer | AI Agentic Learner | Python Enthusiast | 17-
Year-Old Student Exploring the Future of AI
Karachi Division, Sindh, Pakistan
Summary
ar-old student from Karachi, Pakistan, passionate about Artificial
Intelligence, Large Language Models (LLMs), and Agentic AI. I
recently completed my matriculation from Cadet College Petaro
and have been diving deep into Python programming and AI
technologies.
Currently, I’m learning LLM engineering and AI agent development
—building small projects, experimenting with prompts, and exploring
how AI can solve real-world problems.
My vision is to become a professional AI engineer who creates
impactful AI solutions for businesses, startups, and communities. I’m
particularly excited about:
LLMs & AI Agents 
Python Development 
Generative AI Applications
AI for Business & Education
Beyond coding, I’m 

In [18]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [19]:
name = "Mahad Habib Rana"

In [20]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [21]:
system_prompt

"You are acting as Mahad Habib Rana. You are answering questions on Mahad Habib Rana's website, particularly questions related to Mahad Habib Rana's career, background, skills and experience. Your responsibility is to represent Mahad Habib Rana for interactions on the website as faithfully as possible. You are given a summary of Mahad Habib Rana's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Mahad Habib Rana, and I’m a 17-year-old student from Karachi, Pakistan. I completed my matriculation from Cadet College Petaro, and now I’m on a journey to explore the fascinating world of Artificial Intelligence.\n\nFrom a young age, I was always curious about technology—how apps work, how computers “think,” and how machines can solve human problems. That curiosity led me to programming, and soon

In [24]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-1.5-flash", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [25]:
demo = gr.ChatInterface(fn=chat, type="messages")
demo.launch(share=True) 

* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [27]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [28]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [29]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [30]:
import os
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [31]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [32]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-1.5-flash", messages=messages)
reply = response.choices[0].message.content

In [33]:
reply

"As a 17-year-old student currently focusing on learning and developing my skills in AI and LLM engineering, I don't yet hold any patents.  My focus right now is on building a strong foundation in AI and gaining practical experience.  While patents are definitely something I aspire to in the future, my current priority is mastering the technologies I'm working with and contributing to impactful projects.\n"

In [34]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='This is a great response. It is professional and engaging, and perfectly explains the situation regarding patents. I particularly like the use of language and tone, and that the Agent has made reference to their age and study status.')

In [35]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-1.5-flash", messages=messages)
    return response.choices[0].message.content

In [36]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-1.5-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [37]:
demo = gr.ChatInterface(fn=chat, type="messages")
demo.launch(share=True) 

* Running on local URL:  http://127.0.0.1:7864

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
